In [1]:
ONNX_PATH = "LinearBlend_v001.ort"
HEX_CONTENT_PATH = "LinearBlend_v001.txt"

with open(ONNX_PATH, "rb") as fread:
    model_in_bytes = fread.read()
    print("type(model_in_bytes):", type(model_in_bytes))
    model_in_bytes_hex = model_in_bytes.hex()
    print("type(model_in_bytes_hex):", type(model_in_bytes_hex))
    print(len(model_in_bytes_hex))

# Uncomment the following if want to write out hex content to .txt file
# with open(ONNX_PATH, "rb") as fread, open(HEX_CONTENT_PATH, "w") as fwrite:
#     model_in_bytes = fread.read()
#     model_in_bytes_hex = model_in_bytes.hex()
#     fwrite.write(model_in_bytes_hex)

type(model_in_bytes): <class 'bytes'>
type(model_in_bytes_hex): <class 'str'>
6880


In [2]:
import numpy as np
import onnxruntime as ort

with open(HEX_CONTENT_PATH, "r") as fread:
    model_in_bytes_hex = fread.read()
    print(len(model_in_bytes_hex))

model_in_bytes = bytes.fromhex(model_in_bytes_hex)
linear_blend_session = ort.InferenceSession(model_in_bytes, providers=["CPUExecutionProvider"])

img1 = np.random.rand(1, 3, 128, 128).astype(np.float32)
img2 = np.random.rand(1, 3, 128, 128).astype(np.float32)
alpha = np.random.rand(1, 128, 128).astype(np.float32)

result_expected = (1 - np.expand_dims(alpha, 1)) * img1 + np.expand_dims(alpha, 1) * img2
# print("result_expected.shape:", result_expected.shape)
result_actual = linear_blend_session.run(None, {"img1": img1, "img2": img2, "alpha": alpha})[0]
# print("result_actual.shape:", result_actual.shape)

def mse(a, b):
    return np.mean((a-b)**2)

print(f"MSE: {mse(result_expected, result_actual)}")

6880
MSE: 0.0
